In [ ]:
import mlrun
from dotenv import load_dotenv
import os
load_dotenv("secrets.env")
project = mlrun.get_or_create_project("adk-project")

# Create a demo secret for testing
project.set_secrets(secrets={"OPENAI_BASE_URL":os.environ["OPENAI_BASE_URL"], "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"],
"AWS_ACCESS_KEY_ID": os.environ["AWS_ACCESS_KEY_ID"],"AWS_SECRET_ACCESS_KEY": os.environ["AWS_SECRET_ACCESS_KEY"]})

## Log Artifacts
- Log reviews by customers.
- Log sanitation reports by the health inspector.

Please upload the parquet files before logging the artifacts.

In [ ]:
project.log_artifact(
                "restaurants_reviews",
                local_path="v3io://projects/adk-project/restaurants_reviews.pq",
)
project.log_artifact(
                "restaurants_sanitation_reports",
                local_path="s3://bucket_name/path/restaurants_sanitation_reports.pq",
            )

In [ ]:
!tar -czvf /User/v3io/projects/adk-project/agent_demo.tar.gz /User/agent_demo

## Setting up application runtime
- Using a mlrun image with including the requirements for the adk agent.
- Setting up the internal application port.

In [ ]:
application = project.set_function(
    name="adk-app",
    kind="application",
    image = "mlrun/mlrun",
    requirements= ["litellm","google-adk","python-multipart"]
)

In [ ]:
application.with_source_archive(
    "v3io:///projects/adk-project/agent_demo.tar.gz", pull_at_runtime=False
)

In [ ]:
application.set_internal_application_port(5000)

application.spec.command = "bash"
application.spec.args = ["-c","cd /home/mlrun_code/User/agent_demo && adk web --host 127.0.0.1 --port 5000"]

In [ ]:
application.deploy()